In [1]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv("output\German1\DATA_SMOOTH_TRAJECTORIES.csv",names=["time","traj_id","x","y"], delimiter="\t")

In [98]:
list_of_trajid = df.groupby(by=["time"]).traj_id.agg([list,"count"]) #generating the list of traj_ids that is presented in each timeframe

In [99]:
list_of_trajid_gt_1 = list_of_trajid[list_of_trajid["count"]>1] #getting the trajid list which is greater then 1

In [100]:
traj_ids_unique = [list(d) for d in list(set(list_of_trajid_gt_1.list.apply(lambda list_of_trajid_gt_1: tuple(list_of_trajid_gt_1))))]

In [108]:
selected_idx = np.random.choice(np.arange(0,len(traj_ids_unique)), 3)

In [112]:
selected_trajids = [traj_ids_unique[x] for x in selected_idx]

In [113]:
selected_trajids

[[1793, 5730], [7965, 10537, 1800, 9810], [5261, 6520]]